LOAD DATA 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import pandas as pd
from tqdm import tqdm
from matplotlib.transforms import Affine2D
from matplotlib.colors import to_rgba
from tabulate import tabulate

In [ ]:

data_path = 'task2_data'
evaluation_data_path = 'task2_evaluation_data'
cv_csv_path='cv_results.csv'
ca_csv_path='ca_results.csv'
file_count = len(os.listdir(data_path))

print(f"The number of scenarios under {data_path} is: {file_count}")

run_demo = False

In [ ]:
#########Your can use the following code for reference to load the data and visualize the scenario and agents#########
if run_demo:
    for file in tqdm(os.listdir(data_path)):

        info = np.load(data_path+'/'+file,allow_pickle=True)
        all_agent_trajs        = info['all_agent']
        
        # You can run below code to see the shape of the all_agent_trajs
        # print("agents matrix shape is: ",all_agent_trajs.shape)  
        # agents matrix shape is:  (Na, 91, 10)
        # [Na agents, 91 timesteps the interval between each timestep is 0.1 second,\
        #10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
        #the valid flag is either 0 or 1 ; 1 means the valid, 0 means invalid
        obj_types              = info['object_type']
        lane_polylines         = info['lane']           #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
        road_polylines         = info['road_polylines'] #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
        crosswalk_polylines    = info['crosswalk']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
        speed_bump_polylines   = info['speed_bump']     #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
        stop_signs_polylines   = info['stop_sign']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
        drive_way_polylines    = info['drive_way']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
        sdc_track_id           = info['sdc_track_index']   #  the track id of the sdc
        scenario_id            = info['scenario_id']
        
        # We select the 11th timestep as the current timestep to predict the future trajectory of the agents
        all_agent_current = all_agent_trajs[:,10]
        # print(all_agent_current.shape)
        
        #ground truth for the future 3s,5s,8s can be easily derived by :
        predict_horizon = 80
        all_gt_future=all_agent_trajs[:,11:11+predict_horizon]
        all_gt_future.shape
        
        tracks        = info['predict_list']   # the list of agent ids to predict, the agent id is the index in the all_agent_trajs
        tracks
        agents_to_predict = all_agent_trajs[tracks]
        ######################################################
        # The homework should be done on these agents_to_predict.
        # Visualize the agents_to_predict and the scenarios
        # You can use the following code to visualize the scenario
        # For agnets, you can use black color to represent the agents not in the tracks to predict, and use different colors for agents_to_predict
        # You can draw each agent as a rectangle, the center of the rectangle is the center_x and center_y, 
        # and the length and width of the rectangle is boundingbox_x and boundingbox_y
        # We provide a example fig for your reference.   example.png
        sdc_current_state = all_agent_trajs[sdc_track_id][10]
        ax = plt.gca()
        fig = plt.gcf()
        fig.set_facecolor('xkcd:grey') 
        ax.set_facecolor('xkcd:grey')
        for polyline in road_polylines:
            map_type = polyline[0,6]
            if map_type == 6:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
            elif map_type == 7:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
            elif map_type == 8:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
            elif map_type == 9:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
            elif map_type == 10:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
            elif map_type == 11:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
            elif map_type == 12:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
            elif map_type == 13:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
            elif map_type == 15:
                plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
            elif map_type == 16:
                plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
        # 'TYPE_FREEWAY': 1,
        # 'TYPE_SURFACE_STREET': 2,
        # 'TYPE_BIKE_LANE': 3,

        # # for roadline
        # 'TYPE_UNKNOWN': -1,
        # 'TYPE_BROKEN_SINGLE_WHITE': 6,
        # 'TYPE_SOLID_SINGLE_WHITE': 7,
        # 'TYPE_SOLID_DOUBLE_WHITE': 8,
        # 'TYPE_BROKEN_SINGLE_YELLOW': 9,
        # 'TYPE_BROKEN_DOUBLE_YELLOW': 10,
        # 'TYPE_SOLID_SINGLE_YELLOW': 11,
        # 'TYPE_SOLID_DOUBLE_YELLOW': 12,
        # 'TYPE_PASSING_DOUBLE_YELLOW': 13,

        # # for roadedge
        # 'TYPE_ROAD_EDGE_BOUNDARY': 15,
        # 'TYPE_ROAD_EDGE_MEDIAN': 16,

        # # for stopsign
        # 'TYPE_STOP_SIGN': 17,

        # # for crosswalk
        # 'TYPE_CROSSWALK': 18,

        # # for speed bump
        # 'TYPE_SPEED_BUMP': 19,
        
        # # for driveway
        # 'TYPE_DRIVEWAY': 20,
        ax.axis([-70+ sdc_current_state[0], 70+ sdc_current_state[0], -70+ sdc_current_state[1], 70 + sdc_current_state[1]])

        filename = f'visualization/{scenario_id}.png' 
        plt.savefig(filename)
        plt.close()

Q1. Use a constant velocity model to predict the future trajectories of the target agents at 3s, 5s, and 8s. Then, calculate the Average Displacement Error (ADE) and Final Displacement Error (FDE). The results should be averaged across all target agents and scenarios.

In [ ]:
# Code for Q1


class CVMScenarioVisualizer:
    def __init__(self, data_path, save_path='cv_visualization/', time_step=0.1, time_intervals=[3, 5, 8]):
        self.data_path = data_path
        self.save_path = save_path
        self.time_step = time_step
        self.time_intervals = time_intervals
        self.results = {}  # Stores CV model predictions
        self.gt_dict = {}  # Stores ground truth

        # Ensure the save directory exists
        os.makedirs(self.save_path, exist_ok=True)

    def _get_track_colors(self, num_tracks):
        color_palette = ['blue', 'green', 'yellow', 'orange', 'cyan', 'saddlebrown', 'pink', 'gray']
        return color_palette[:num_tracks]

    def process_scenarios(self, visualize=True):
        npz_files = [file for file in os.listdir(self.data_path) if file.endswith('.npz')]
        for file in tqdm(npz_files, desc="Processing NPZ Files"):
            file_path = os.path.join(self.data_path, file)
            info = np.load(file_path, allow_pickle=True)
            scenario_id = str(info['scenario_id'])
            self.results[scenario_id], self.gt_dict[scenario_id] = self._process_single_scenario(info, visualize)
        return self.results, self.gt_dict

    def _process_single_scenario(self, info, visualize):
        scenario_results = {}

        all_agent_trajs = info['all_agent']
        all_gt_future = all_agent_trajs[:, 11:11 + 80]
        sdc_track_id = info['sdc_track_index']
        tracks = info['predict_list']
        agents_to_predict = all_agent_trajs[tracks]
        sdc_current_state = all_agent_trajs[sdc_track_id][10]

        track_colors = self._get_track_colors(len(tracks))
        
        for i, (track_id, agent) in enumerate(zip(tracks, agents_to_predict[:, 10])):
            track_id = int(track_id)
            scenario_results[track_id] = {"CV_track": {}, "color": track_colors[i]}
            center_x, center_y, vel_x, vel_y, valid = agent[0], agent[1], agent[7], agent[8], agent[9]
            scenario_results[track_id]["CV_track"]["vel"] = (vel_x, vel_y)
            if valid == 1:
                for t in self.time_intervals:
                    scenario_results[track_id]["CV_track"][t] = (center_x + vel_x * t, center_y + vel_y * t)

        scenario_results["sdc"] = {"CV_sdc": {}, "color": "red"}
        center_x, center_y, vel_x, vel_y, valid = sdc_current_state[0], sdc_current_state[1], sdc_current_state[7], sdc_current_state[8], sdc_current_state[9]
        scenario_results["sdc"]["CV_sdc"]["vel"] = (vel_x, vel_y)
        scenario_results["sdc"]["CV_sdc"]["id"] = int(sdc_track_id)
        for t in self.time_intervals:
            scenario_results["sdc"]["CV_sdc"][t] = (center_x + vel_x * t, center_y + vel_y * t)

        if visualize:
            self._visualize_scenario(info, scenario_results, tracks, sdc_track_id, all_gt_future)
        
        return scenario_results, all_gt_future

    def _visualize_scenario(self, info, scenario_results, tracks, sdc_track_id, all_gt_future):
        scenario_id = str(info['scenario_id'])
        all_agent_current = info['all_agent'][:, 10]
        sdc_current_state = info['all_agent'][info['sdc_track_index']][10]

        road_polylines = info['road_polylines']
        lane_polylines = info['lane']
        crosswalk_polylines = info['crosswalk']
        speed_bump_polylines = info['speed_bump']
        stop_signs_polylines = info['stop_sign']
        drive_way_polylines = info['drive_way']

        
        fig, ax = plt.subplots(figsize=(10, 10))
        fig.set_facecolor('xkcd:grey')
        ax.set_facecolor('xkcd:grey')

        linewidths = {3: 3.0, 5: 2.0, 8: 1.0}

        # Plot road polylines with predefined map types
        for polyline in road_polylines:
            map_type = polyline[0,6]
            if map_type == 6:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
            elif map_type == 7:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
            elif map_type == 8:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
            elif map_type == 9:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
            elif map_type == 10:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
            elif map_type == 11:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
            elif map_type == 12:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
            elif map_type == 13:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
            elif map_type == 15:
                plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
            elif map_type == 16:
                plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
        
        # Draw special markers for additional elements
        for polyline in crosswalk_polylines:
            points = [(point[0], point[1]) for point in polyline]
            ax.add_patch(plt.Polygon(points, closed=True, edgecolor='b', facecolor='none', linewidth=1))
        for polyline in speed_bump_polylines:
            points = [(point[0], point[1]) for point in polyline]
            ax.add_patch(plt.Polygon(points, closed=True, edgecolor='gold', facecolor='none', linewidth=1))
        for polyline in stop_signs_polylines:
            for point in polyline:
                ax.add_patch(plt.Circle((point[0], point[1]), 1, color='red', alpha=0.8))
        # for polyline in lane_polylines:
        #     plt.plot(polyline[:, 0], polyline[:, 1], 'gray', linestyle='dashed', linewidth=1)
        for polyline in drive_way_polylines:
            points = [(point[0], point[1]) for point in polyline]
            ax.add_patch(plt.Polygon(points, closed=True, edgecolor='lightyellow', facecolor='none', linewidth=1))
        
        # Plot tracks' ground truth
        for id in tracks:
            gt_points = all_gt_future[id][:]
            mask = ~((gt_points[:, 0] == 0) & (gt_points[:, 1] == 0))
            filtered_points = gt_points[mask]
            plt.plot(filtered_points[:, 0], filtered_points[:, 1], scenario_results[id]["color"], linestyle='dashed', linewidth=1)
            ax.add_patch(plt.Circle((filtered_points[-1, 0], filtered_points[-1, 1]), 0.5, color=scenario_results[id]["color"], alpha=0.8))

        # Plot self-driving car's (SDC) ground truth
        sdc_points = all_gt_future[sdc_track_id][:]
        plt.plot(sdc_points[:, 0], sdc_points[:, 1], 'purple', linestyle='dashed', linewidth=1)
        ax.add_patch(plt.Circle((sdc_points[-1, 0], sdc_points[-1, 1]), 0.5, color='darkred', alpha=0.8))

        # Plot agents' current positions
        for i, agent in enumerate(all_agent_current):
            x, y, w, h, heading, valid = agent[0], agent[1], agent[3], agent[4], np.degrees(agent[6]), agent[9]
            if valid == 1:
                color = scenario_results[i]["color"] if i in scenario_results else 'black'
                rect = plt.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor=color, alpha=0.5)
                t = Affine2D().rotate_deg_around(x, y, heading) + ax.transData
                rect.set_transform(t)
                ax.add_patch(rect)
        
        # Plot the self-driving car (SDC) with heading rotation
        x, y, w, h, heading, valid = sdc_current_state[0], sdc_current_state[1], sdc_current_state[3], sdc_current_state[4], np.degrees(sdc_current_state[6]), sdc_current_state[9]
        color = 'r'
        rect = plt.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor=color, alpha=0.5)
        t = Affine2D().rotate_deg_around(x, y, heading) + ax.transData

        rect.set_transform(t)
        ax.add_patch(rect)

        # Plot CV prediction arrows (tracks)
        for track_id, track_data in scenario_results.items():
            if track_id == "sdc":
                continue
            x_start, y_start = all_agent_current[track_id][0], all_agent_current[track_id][1]
            base_color = track_data["color"]
            for t in self.time_intervals:
                if t in track_data["CV_track"]:
                    x_end, y_end = track_data["CV_track"][t]
                    alpha = {3: 1.0, 5: 0.7, 8: 0.4}[t]
                    shaded_color = to_rgba(base_color, alpha)
                    ax.arrow(x_start, y_start, x_end - x_start, y_end - y_start,
                            head_width=0.5, head_length=1.0, fc=shaded_color, ec=shaded_color, linewidth=linewidths[t])

        # Plot CV prediction arrows (sdc)
        x_start, y_start = all_agent_current[sdc_track_id][0], all_agent_current[sdc_track_id][1]
        for t in self.time_intervals:
            if t in scenario_results["sdc"]["CV_sdc"]:
                x_end, y_end = scenario_results["sdc"]["CV_sdc"][t]
                ax.arrow(x_start, y_start, x_end - x_start, y_end - y_start,
                         head_width=0.5, head_length=1.0, fc='red', ec='red', linewidth=linewidths[t])


        ax.axis([-70+ sdc_current_state[0], 70+ sdc_current_state[0], -70+ sdc_current_state[1], 70 + sdc_current_state[1]])
        plt.savefig(os.path.join(self.save_path, f'{scenario_id}.png'))
        plt.close()


class CVTrajectoryEvaluator:
    def __init__(self, time_step=0.1, time_intervals=[3, 5, 8]):
        self.time_step = time_step
        self.time_intervals = time_intervals
        self.time_frames = {t: int(t / self.time_step - 1) for t in self.time_intervals}

    def compute_fde(self, pred_data, gt_data, t):
        gt_x_t, gt_y_t = gt_data[self.time_frames[t]][:2]

        if gt_x_t == 0 or gt_y_t == 0:
            return np.nan, np.nan
        
        return np.linalg.norm(np.array([pred_data[t][0], pred_data[t][1]]) - np.array([gt_x_t, gt_y_t])), np.array([pred_data[t][0], pred_data[t][1]]) - np.array([gt_x_t, gt_y_t])

    def compute_ade(self, pred_data, gt_data, t):
        gt_positions = np.array([gt_data[step][:2] for step in range(0, self.time_frames[t] + 1)])
        mask = ~((gt_positions[:, 0] == 0) & (gt_positions[:, 1] == 0))
        valid_gt_positions = gt_positions[mask]

        if valid_gt_positions.shape[0] == 0:
            return np.nan

        vel_x, vel_y = pred_data["vel"]
        pred_positions = np.array([
            [pred_data[t][0] - vel_x * (t - step * self.time_step),
             pred_data[t][1] - vel_y * (t - step * self.time_step)]
            for step in range(1, self.time_frames[t] + 2)
        ])
        valid_pred_positions = pred_positions[mask]

        return np.mean(np.linalg.norm(valid_pred_positions - valid_gt_positions, axis=1)), np.linalg.norm(valid_pred_positions - valid_gt_positions, axis=1)

    def compute_ade_fde(self, results, ground_truths, save_csv_path='results.csv'):
        data = []

        for scenario_id, scenario_data in results.items():
            if scenario_id not in ground_truths:
                continue

            ground_truth = ground_truths[scenario_id]

            # Initialize storage for ADE & FDE results
            ade_tracks, fde_tracks = {t: [] for t in self.time_intervals}, {t: [] for t in self.time_intervals}
            ade_sdc, fde_sdc = {t: [] for t in self.time_intervals}, {t: [] for t in self.time_intervals}

            # Compute ADE & FDE for all tracks (except SDC)
            for track_id, track_data in scenario_data.items():
                if track_id == "sdc":
                    continue

                for t in self.time_intervals:
                    if "vel" in track_data["CV_track"] and t in track_data["CV_track"]:
                        ade, _ = self.compute_ade(track_data["CV_track"], ground_truth[track_id], t)
                        fde, _ = self.compute_fde(track_data["CV_track"], ground_truth[track_id], t)
                        ade_tracks[t].append(ade)
                        fde_tracks[t].append(fde)

            # Compute ADE & FDE for SDC
            if "sdc" in scenario_data and "CV_sdc" in scenario_data["sdc"]:
                sdc_data = scenario_data["sdc"]["CV_sdc"]
                sdc_id = sdc_data["id"]

                for t in self.time_intervals:
                    if "vel" in sdc_data and t in sdc_data:
                        ade, _ = self.compute_ade(sdc_data, ground_truth[sdc_id], t)
                        fde, _ = self.compute_fde(sdc_data, ground_truth[sdc_id], t)
                        ade_sdc[t].append(ade)
                        fde_sdc[t].append(fde)

            # Store results for this scenario
            row = [scenario_id]
            row += [np.nanmean(ade_tracks[t]) if ade_tracks[t] else np.nan for t in self.time_intervals]
            row += [np.nanmean(ade_sdc[t]) if ade_sdc[t] else np.nan for t in self.time_intervals]
            row += [np.nanmean(fde_tracks[t]) if fde_tracks[t] else np.nan for t in self.time_intervals]
            row += [np.nanmean(fde_sdc[t]) if fde_sdc[t] else np.nan for t in self.time_intervals]

            data.append(row)

        # Create DataFrame with the required headers
        columns = ["scenario"] + \
                  [f"ADE tracks ({t}s)" for t in self.time_intervals] + \
                  [f"ADE sdc ({t}s)" for t in self.time_intervals] + \
                  [f"FDE tracks ({t}s)" for t in self.time_intervals] + \
                  [f"FDE sdc ({t}s)" for t in self.time_intervals]

        df = pd.DataFrame(data, columns=columns)
        df.to_csv(save_csv_path, index=False)
        print(f"Results saved to {save_csv_path}")

    def plot_evaluate_result(self, results, ground_truths, save_path):
        for t in self.time_intervals:
            plt.figure(figsize=(12, 6))
            for scenario_id, scenario_data in results.items():
                ground_truth = ground_truths.get(scenario_id, None)
                if ground_truth is None or ground_truth.size == 0:
                    continue

                for track_id, track_data in scenario_data.items():
                    if track_id == "sdc":
                        label_id = "SDC"
                        continue
                    else:
                        label_id = f"Agent {track_id}"
                    ade_mean, ade_list = self.compute_ade(track_data["CV_track"], ground_truth[track_id], t)
                    fde, _ = self.compute_fde(track_data["CV_track"], ground_truth[track_id], t)
                    time_steps = range(1, len(ade_list) + 1)

                    plt.plot(time_steps, ade_list, label=f"{label_id}, average ADE: {ade_mean:.2f}, FDE: {fde:.2f}")

                sdc_data = scenario_data["sdc"]["CV_sdc"]
                sdc_id = sdc_data["id"]
                label_id = f"SDC {sdc_id}"
                ade_mean, ade_list = self.compute_ade(sdc_data, ground_truth[sdc_id], t)
                fde, _ = self.compute_fde(sdc_data, ground_truth[sdc_id], t)
                time_steps = range(1, len(ade_list) + 1)

                plt.plot(time_steps, ade_list, label=f"{label_id}, average ADE: {ade_mean:.5f}, FDE: {fde:.5f}")

            plt.title(f"Displacement Errors Using CV Model in Future {t}s")
            plt.xlabel("Time Step")
            plt.ylabel("Displacement Error")
            plt.legend()
            plt.grid(True)
            plt.savefig(f"{save_path}/Displacement Errors Using CV Model in Future {t}s", dpi=300, bbox_inches='tight')


if __name__ == "__main__":
    model = CVMScenarioVisualizer(data_path)

    results, ground_truths = model.process_scenarios()

    evaluator = CVTrajectoryEvaluator()
    evaluator.compute_ade_fde(results, ground_truths, save_csv_path=cv_csv_path)

In [ ]:
# Load the CSV file
df = pd.read_csv(cv_csv_path)

# Define the columns of interest
columns = [
    "ADE tracks (3s)", "ADE tracks (5s)", "ADE tracks (8s)", 
    "ADE sdc (3s)", "ADE sdc (5s)", "ADE sdc (8s)",
    "FDE tracks (3s)", "FDE tracks (5s)", "FDE tracks (8s)",
    "FDE sdc (3s)", "FDE sdc (5s)", "FDE sdc (8s)"
]

# Convert data to numeric, forcing errors to NaN
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')

# Calculate mean values, ignoring NaNs
means = df[columns].mean(skipna=True)

# Prepare tabulated data
table_data = [
    ["ADE", means["ADE tracks (3s)"], means["ADE tracks (5s)"], means["ADE tracks (8s)"],
           means["ADE sdc (3s)"], means["ADE sdc (5s)"], means["ADE sdc (8s)"]],
    ["FDE", means["FDE tracks (3s)"], means["FDE tracks (5s)"], means["FDE tracks (8s)"],
           means["FDE sdc (3s)"], means["FDE sdc (5s)"], means["FDE sdc (8s)"]]
]

# Print the results in a formatted table
headers = ["CVM", "Tracks (3s)", "Tracks (5s)", "Tracks (8s)", "SDC (3s)", "SDC (5s)", "SDC (8s)"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Define the columns for ADE and FDE
ade_tracks = ["ADE tracks (3s)", "ADE tracks (5s)", "ADE tracks (8s)"]
ade_sdc = ["ADE sdc (3s)", "ADE sdc (5s)", "ADE sdc (8s)"]
fde_tracks = ["FDE tracks (3s)", "FDE tracks (5s)", "FDE tracks (8s)"]
fde_sdc = ["FDE sdc (3s)", "FDE sdc (5s)", "FDE sdc (8s)"]

# Convert data to numeric, handling NaNs
df[ade_tracks + ade_sdc + fde_tracks + fde_sdc] = df[ade_tracks + ade_sdc + fde_tracks + fde_sdc].apply(pd.to_numeric, errors='coerce')

# Prepare data for box plot
ade_tracks_data = [df[col].dropna() for col in ade_tracks]
ade_sdc_data = [df[col].dropna() for col in ade_sdc]
fde_tracks_data = [df[col].dropna() for col in fde_tracks]
fde_sdc_data = [df[col].dropna() for col in fde_sdc]

# Create Box Plot
plt.figure(figsize=(10, 6))

# Limit Y-axis range from 0 to 80
# plt.ylim(-5, 200)

# Box plot positions
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Plot ADE Tracks (Blue)
plt.boxplot(ade_tracks_data, positions=[1, 5, 9], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="blue", alpha=0.5), medianprops=dict(color="black"))

# Plot ADE SDC (Light Blue)
plt.boxplot(ade_sdc_data, positions=[2, 6, 10], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="skyblue", alpha=0.5), medianprops=dict(color="black"))

# Plot FDE Tracks (Orange)
plt.boxplot(fde_tracks_data, positions=[3, 7, 11], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="red", alpha=0.5), medianprops=dict(color="black"))

# Plot FDE SDC (Light Orange)
plt.boxplot(fde_sdc_data, positions=[4, 8, 12], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="pink", alpha=0.5), medianprops=dict(color="black"))

# Set labels
plt.xticks([2.5, 6.5, 10.5], ["3s", "5s", "8s"])  # Group positions centered
plt.xlabel("Time After 1th second (s)")
plt.ylabel("Error Value")
plt.title("Constant Velocity Model Prediction Error (ADE & FDE)")

# Add legend with color distinction
plt.legend([
    plt.Rectangle((0, 0), 1, 1, fc="blue", alpha=0.5),
    plt.Rectangle((0, 0), 1, 1, fc="skyblue", alpha=0.5),
    plt.Rectangle((0, 0), 1, 1, fc="red", alpha=0.5),
    plt.Rectangle((0, 0), 1, 1, fc="pink", alpha=0.5)
], ["ADE Tracks", "ADE SDC", "FDE Tracks", "FDE SDC"], loc="upper left")

plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


Q2. Use a constant acceleration model to predict the future trajectories of the target agents at 3s, 5s, and 8s. Then, calculate the Average Displacement Error (ADE) and Final Displacement Error (FDE). The results should be averaged across all target agents and scenarios.

In [ ]:
# Code for Q2

class CAMScenarioVisualizer:
    def __init__(self, data_path, save_path='ca_visualization/', time_step=0.1, time_intervals=[3, 5, 8]):
        self.data_path = data_path
        self.save_path = save_path
        self.time_step = time_step
        self.time_intervals = time_intervals
        self.results = {}  # Stores CV model predictions
        self.gt_dict = {}  # Stores ground truth

        # Ensure the save directory exists
        os.makedirs(self.save_path, exist_ok=True)

    def _get_track_colors(self, num_tracks):
        color_palette = ['blue', 'green', 'yellow', 'orange', 'cyan', 'saddlebrown', 'pink', 'gray']
        return color_palette[:num_tracks]

    def process_scenarios(self, visualize=True):
        npz_files = [file for file in os.listdir(self.data_path) if file.endswith('.npz')]
        for file in tqdm(npz_files, desc="Processing NPZ Files"):
            file_path = os.path.join(self.data_path, file)
            info = np.load(file_path, allow_pickle=True)
            scenario_id = str(info['scenario_id'])
            self.results[scenario_id], self.gt_dict[scenario_id] = self._process_single_scenario(info, visualize)
        return self.results, self.gt_dict

    def _process_single_scenario(self, info, visualize):
        scenario_results = {}

        all_agent_trajs = info['all_agent']
        all_gt_future = all_agent_trajs[:, 11:11 + 80]
        sdc_track_id = info['sdc_track_index']
        tracks = info['predict_list']
        agents_to_predict = all_agent_trajs[tracks]
        sdc_current_state = all_agent_trajs[sdc_track_id][9:11]

        track_colors = self._get_track_colors(len(tracks))
        
        for i, (track_id, agent_states) in enumerate(zip(tracks, agents_to_predict[:, 9:11])):
            track_id = int(track_id)
            scenario_results[track_id] = {"CV_track": [], "color": track_colors[i]}
            
            prev_state, curr_state = agent_states[0], agent_states[1]
            center_x, center_y = curr_state[0], curr_state[1]
            prev_vel_x, prev_vel_y = prev_state[7], prev_state[8]
            vel_x, vel_y = curr_state[7], curr_state[8]
            valid = curr_state[9]
            
            if valid == 1:
                acc_x = (vel_x - prev_vel_x) / self.time_step
                acc_y = (vel_y - prev_vel_y) / self.time_step
                trajectory = []

                for t in range(1, max(self.time_intervals) * 10 + 1):
                    center_x = center_x + vel_x * self.time_step + 0.5 * acc_x * self.time_step ** 2
                    center_y = center_y + vel_y * self.time_step + 0.5 * acc_y * self.time_step ** 2
                    vel_x = vel_x + acc_x * self.time_step
                    vel_y = vel_y + acc_y * self.time_step

                    trajectory.append((center_x, center_y))

                scenario_results[track_id]["CV_track"] = trajectory

        # SDC
        scenario_results["sdc"] = {"CV_sdc": [], "id": int(sdc_track_id), "color": "red"}

        prev_state, curr_state = sdc_current_state[0], sdc_current_state[1]
        center_x, center_y = curr_state[0], curr_state[1]
        prev_vel_x, prev_vel_y = prev_state[7], prev_state[8]
        vel_x, vel_y = curr_state[7], curr_state[8]

        acc_x = (vel_x - prev_vel_x) / self.time_step
        acc_y = (vel_y - prev_vel_y) / self.time_step
        trajectory = []

        for t in range(1, max(self.time_intervals) * 10 + 1):
            center_x = center_x + vel_x * self.time_step + 0.5 * acc_x * self.time_step ** 2
            center_y = center_y + vel_y * self.time_step + 0.5 * acc_y * self.time_step ** 2
            vel_x = vel_x + acc_x * self.time_step
            vel_y = vel_y + acc_y * self.time_step
            trajectory.append((center_x, center_y))

        scenario_results["sdc"]["CV_sdc"] = trajectory

        if visualize:
            self._visualize_scenario(info, scenario_results, tracks, sdc_track_id, all_gt_future)
        
        return scenario_results, all_gt_future

    def _visualize_scenario(self, info, scenario_results, tracks, sdc_track_id, all_gt_future):
        scenario_id = str(info['scenario_id'])
        all_agent_current = info['all_agent'][:, 10]
        sdc_current_state = info['all_agent'][info['sdc_track_index']][10]

        road_polylines = info['road_polylines']
        lane_polylines = info['lane']
        crosswalk_polylines = info['crosswalk']
        speed_bump_polylines = info['speed_bump']
        stop_signs_polylines = info['stop_sign']
        drive_way_polylines = info['drive_way']

        
        fig, ax = plt.subplots(figsize=(10, 10))
        fig.set_facecolor('xkcd:grey')
        ax.set_facecolor('xkcd:grey')

        linewidths = {3: 3.0, 5: 2.0, 8: 1.0}

        # Plot road polylines with predefined map types
        for polyline in road_polylines:
            map_type = polyline[0,6]
            if map_type == 6:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
            elif map_type == 7:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
            elif map_type == 8:
                plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
            elif map_type == 9:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
            elif map_type == 10:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
            elif map_type == 11:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
            elif map_type == 12:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
            elif map_type == 13:
                plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
            elif map_type == 15:
                plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
            elif map_type == 16:
                plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
        
        # Draw special markers for additional elements
        for polyline in crosswalk_polylines:
            points = [(point[0], point[1]) for point in polyline]
            ax.add_patch(plt.Polygon(points, closed=True, edgecolor='b', facecolor='none', linewidth=1))
        for polyline in speed_bump_polylines:
            points = [(point[0], point[1]) for point in polyline]
            ax.add_patch(plt.Polygon(points, closed=True, edgecolor='gold', facecolor='none', linewidth=1))
        for polyline in stop_signs_polylines:
            for point in polyline:
                ax.add_patch(plt.Circle((point[0], point[1]), 1, color='red', alpha=0.8))
        # for polyline in lane_polylines:
        #     plt.plot(polyline[:, 0], polyline[:, 1], 'gray', linestyle='dashed', linewidth=1)
        for polyline in drive_way_polylines:
            points = [(point[0], point[1]) for point in polyline]
            ax.add_patch(plt.Polygon(points, closed=True, edgecolor='lightyellow', facecolor='none', linewidth=1))
        
        # Plot tracks' ground truth
        for id in tracks:
            gt_points = all_gt_future[id][:]
            mask = ~((gt_points[:, 0] == 0) & (gt_points[:, 1] == 0))
            filtered_points = gt_points[mask]
            plt.plot(filtered_points[:, 0], filtered_points[:, 1], scenario_results[id]["color"], linestyle='dashed', linewidth=1)
            ax.add_patch(plt.Circle((filtered_points[-1, 0], filtered_points[-1, 1]), 0.5, color=scenario_results[id]["color"], alpha=0.8))

        # Plot self-driving car's (SDC) ground truth
        sdc_points = all_gt_future[sdc_track_id][:]
        plt.plot(sdc_points[:, 0], sdc_points[:, 1], 'purple', linestyle='dashed', linewidth=1)
        ax.add_patch(plt.Circle((sdc_points[-1, 0], sdc_points[-1, 1]), 0.5, color='darkred', alpha=0.8))

        # Plot agents' current positions
        for i, agent in enumerate(all_agent_current):
            x, y, w, h, heading, valid = agent[0], agent[1], agent[3], agent[4], np.degrees(agent[6]), agent[9]
            if valid == 1:
                color = scenario_results[i]["color"] if i in scenario_results else 'black'
                rect = plt.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor=color, alpha=0.5)
                t = Affine2D().rotate_deg_around(x, y, heading) + ax.transData
                rect.set_transform(t)
                ax.add_patch(rect)
        
        # Plot the self-driving car (SDC) with heading rotation
        x, y, w, h, heading, valid = sdc_current_state[0], sdc_current_state[1], sdc_current_state[3], sdc_current_state[4], np.degrees(sdc_current_state[6]), sdc_current_state[9]
        color = 'r'
        rect = plt.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor=color, alpha=0.5)
        t = Affine2D().rotate_deg_around(x, y, heading) + ax.transData

        rect.set_transform(t)
        ax.add_patch(rect)

        # Plot CV prediction arrows (tracks)
        for track_id, track_data in scenario_results.items():
            if track_id == "sdc":
                continue
            base_color = track_data["color"]
            for t in self.time_intervals:
                x_values = [x[0] for x in track_data["CV_track"][:t * 10]]
                y_values = [y[1] for y in track_data["CV_track"][:t * 10]]
                alpha = {3: 1.0, 5: 0.7, 8: 0.4}[t]
                shaded_color = to_rgba(base_color, alpha)
                plt.plot(x_values, y_values, linestyle='-', marker='', color=shaded_color, linewidth=linewidths[t])
                ax.arrow(x_values[-2], y_values[-2], x_values[-1] - x_values[-2], y_values[-1] - y_values[-2],
                     head_width=0.5, head_length=1.0, fc=shaded_color, ec=shaded_color, linewidth=linewidths[t])

        # Plot CV prediction arrows (sdc)
        for t in self.time_intervals:
                x_values = [x[0] for x in scenario_results["sdc"]["CV_sdc"][:t * 10]]
                y_values = [y[1] for y in scenario_results["sdc"]["CV_sdc"][:t * 10]]
                plt.plot(x_values, y_values, linestyle='-', marker='', color='red', linewidth=linewidths[t])
                ax.arrow(x_values[-2], y_values[-2], x_values[-1] - x_values[-2], y_values[-1] - y_values[-2],
                         head_width=0.5, head_length=1.0, fc='red', ec='red', linewidth=linewidths[t])

        ax.axis([-70+ sdc_current_state[0], 70+ sdc_current_state[0], -70+ sdc_current_state[1], 70 + sdc_current_state[1]])
        plt.savefig(os.path.join(self.save_path, f'{scenario_id}.png'))
        plt.close()


class CATrajectoryEvaluator:
    def __init__(self, time_step=0.1, time_intervals=[3, 5, 8]):
        self.time_step = time_step
        self.time_intervals = time_intervals
        self.time_frames = {t: int(t / self.time_step - 1) for t in self.time_intervals}

    def compute_fde(self, pred_data, gt_data, t):
        gt_x_t, gt_y_t = gt_data[self.time_frames[t]][:2]

        if gt_x_t == 0 or gt_y_t == 0:
            return np.nan
        
        return np.linalg.norm(np.array([pred_data[self.time_frames[t]][0], pred_data[self.time_frames[t]][1]]) - np.array([gt_x_t, gt_y_t]))

    def compute_ade(self, pred_data, gt_data, t):
        gt_positions = np.array([gt_data[step][:2] for step in range(0, self.time_frames[t] + 1)])
        mask = ~((gt_positions[:, 0] == 0) & (gt_positions[:, 1] == 0))
        valid_gt_positions = gt_positions[mask]

        if valid_gt_positions.shape[0] == 0:
            return np.nan, np.nan

        pred_positions = np.array(pred_data[:self.time_frames[t] + 1])
        valid_pred_positions = pred_positions[mask]

        return np.mean(np.linalg.norm(valid_pred_positions - valid_gt_positions, axis=1)), np.linalg.norm(valid_pred_positions - valid_gt_positions, axis=1)

    def compute_ade_fde(self, results, ground_truths, save_csv_path='results.csv'):
        data = []

        for scenario_id, scenario_data in results.items():
            if scenario_id not in ground_truths:
                continue

            ground_truth = ground_truths[scenario_id]

            # Initialize storage for ADE & FDE results
            ade_tracks, fde_tracks = {t: [] for t in self.time_intervals}, {t: [] for t in self.time_intervals}
            ade_sdc, fde_sdc = {t: [] for t in self.time_intervals}, {t: [] for t in self.time_intervals}

            # Compute ADE & FDE for all tracks (except SDC)
            for track_id, track_data in scenario_data.items():
                if track_id == "sdc":
                    continue

                for t in self.time_intervals:
                    ade, _ = self.compute_ade(track_data["CV_track"], ground_truth[track_id], t)
                    fde= self.compute_fde(track_data["CV_track"], ground_truth[track_id], t)
                    ade_tracks[t].append(ade)
                    fde_tracks[t].append(fde)

            # Compute ADE & FDE for SDC
            if "sdc" in scenario_data and "CV_sdc" in scenario_data["sdc"]:
                sdc_data = scenario_data["sdc"]["CV_sdc"]
                sdc_id = scenario_data["sdc"]["id"]

                for t in self.time_intervals:
                    ade, _ = self.compute_ade(sdc_data, ground_truth[sdc_id], t)
                    fde = self.compute_fde(sdc_data, ground_truth[sdc_id], t)
                    ade_sdc[t].append(ade)
                    fde_sdc[t].append(fde)

            # Store results for this scenario
            row = [scenario_id]
            row += [np.nanmean(ade_tracks[t]) if ade_tracks[t] else np.nan for t in self.time_intervals]
            row += [np.nanmean(ade_sdc[t]) if ade_sdc[t] else np.nan for t in self.time_intervals]
            row += [np.nanmean(fde_tracks[t]) if fde_tracks[t] else np.nan for t in self.time_intervals]
            row += [np.nanmean(fde_sdc[t]) if fde_sdc[t] else np.nan for t in self.time_intervals]

            data.append(row)

        # Create DataFrame with the required headers
        columns = ["scenario"] + \
                  [f"ADE tracks ({t}s)" for t in self.time_intervals] + \
                  [f"ADE sdc ({t}s)" for t in self.time_intervals] + \
                  [f"FDE tracks ({t}s)" for t in self.time_intervals] + \
                  [f"FDE sdc ({t}s)" for t in self.time_intervals]

        df = pd.DataFrame(data, columns=columns)
        df.to_csv(save_csv_path, index=False)
        print(f"Results saved to {save_csv_path}")

    def plot_evaluate_result(self, results, ground_truths, save_path):
        for t in self.time_intervals:
            plt.figure(figsize=(12, 6))
            for scenario_id, scenario_data in results.items():
                ground_truth = ground_truths.get(scenario_id, None)
                if ground_truth is None or ground_truth.size == 0:
                    continue

                for track_id, track_data in scenario_data.items():
                    if track_id == "sdc":
                        label_id = "SDC"
                        continue
                    else:
                        label_id = f"Agent {track_id}"
                    ade_mean, ade_list = self.compute_ade(track_data["CV_track"], ground_truth[track_id], t)
                    fde = self.compute_fde(track_data["CV_track"], ground_truth[track_id], t)
                    time_steps = range(1, len(ade_list) + 1)

                    plt.plot(time_steps, ade_list, label=f"{label_id}, average ADE: {ade_mean:.2f}, FDE: {fde:.2f}")

                sdc_data = scenario_data["sdc"]["CV_sdc"]
                sdc_id = scenario_data["sdc"]["id"]
                label_id = f"SDC {sdc_id}"
                ade_mean, ade_list = self.compute_ade(sdc_data, ground_truth[sdc_id], t)
                fde = self.compute_fde(sdc_data, ground_truth[sdc_id], t)
                time_steps = range(1, len(ade_list) + 1)

                plt.plot(time_steps, ade_list, label=f"{label_id}, average ADE: {ade_mean:.5f}, FDE: {fde:.5f}")

            plt.title(f"Displacement Errors Using CA Model in Future {t}s")
            plt.xlabel("Time Step")
            plt.ylabel("Displacement Error")
            plt.legend()
            plt.grid(True)
            plt.savefig(f"{save_path}/Displacement Errors Using CA Model in Future {t}s", dpi=300, bbox_inches='tight')


if __name__ == "__main__":
    model = CAMScenarioVisualizer(data_path)

    results, ground_truths = model.process_scenarios()

    evaluator = CATrajectoryEvaluator()
    evaluator.compute_ade_fde(results, ground_truths, save_csv_path=ca_csv_path)

In [ ]:
# Load the CSV file
df = pd.read_csv(ca_csv_path)

# Define the columns of interest
columns = [
    "ADE tracks (3s)", "ADE tracks (5s)", "ADE tracks (8s)", 
    "ADE sdc (3s)", "ADE sdc (5s)", "ADE sdc (8s)",
    "FDE tracks (3s)", "FDE tracks (5s)", "FDE tracks (8s)",
    "FDE sdc (3s)", "FDE sdc (5s)", "FDE sdc (8s)"
]

# Convert data to numeric, forcing errors to NaN
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')

# Calculate mean values, ignoring NaNs
means = df[columns].mean(skipna=True)

# Prepare tabulated data
table_data = [
    ["ADE", means["ADE tracks (3s)"], means["ADE tracks (5s)"], means["ADE tracks (8s)"],
           means["ADE sdc (3s)"], means["ADE sdc (5s)"], means["ADE sdc (8s)"]],
    ["FDE", means["FDE tracks (3s)"], means["FDE tracks (5s)"], means["FDE tracks (8s)"],
           means["FDE sdc (3s)"], means["FDE sdc (5s)"], means["FDE sdc (8s)"]]
]

# Print the results in a formatted table
headers = ["CAM", "Tracks (3s)", "Tracks (5s)", "Tracks (8s)", "SDC (3s)", "SDC (5s)", "SDC (8s)"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Define the columns for ADE and FDE
ade_tracks = ["ADE tracks (3s)", "ADE tracks (5s)", "ADE tracks (8s)"]
ade_sdc = ["ADE sdc (3s)", "ADE sdc (5s)", "ADE sdc (8s)"]
fde_tracks = ["FDE tracks (3s)", "FDE tracks (5s)", "FDE tracks (8s)"]
fde_sdc = ["FDE sdc (3s)", "FDE sdc (5s)", "FDE sdc (8s)"]

# Convert data to numeric, handling NaNs
df[ade_tracks + ade_sdc + fde_tracks + fde_sdc] = df[ade_tracks + ade_sdc + fde_tracks + fde_sdc].apply(pd.to_numeric, errors='coerce')

# Prepare data for box plot
ade_tracks_data = [df[col].dropna() for col in ade_tracks]
ade_sdc_data = [df[col].dropna() for col in ade_sdc]
fde_tracks_data = [df[col].dropna() for col in fde_tracks]
fde_sdc_data = [df[col].dropna() for col in fde_sdc]

# Create Box Plot
plt.figure(figsize=(10, 6))

# Limit Y-axis range from 0 to 80
# plt.ylim(-5, 200)

# Box plot positions
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Plot ADE Tracks (Blue)
plt.boxplot(ade_tracks_data, positions=[1, 5, 9], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="blue", alpha=0.5), medianprops=dict(color="black"))

# Plot ADE SDC (Light Blue)
plt.boxplot(ade_sdc_data, positions=[2, 6, 10], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="skyblue", alpha=0.5), medianprops=dict(color="black"))

# Plot FDE Tracks (Orange)
plt.boxplot(fde_tracks_data, positions=[3, 7, 11], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="red", alpha=0.5), medianprops=dict(color="black"))

# Plot FDE SDC (Light Orange)
plt.boxplot(fde_sdc_data, positions=[4, 8, 12], widths=0.3, patch_artist=True,
            boxprops=dict(facecolor="pink", alpha=0.5), medianprops=dict(color="black"))

# Set labels
plt.xticks([2.5, 6.5, 10.5], ["3s", "5s", "8s"])  # Group positions centered
plt.xlabel("Time After 1th second (s)")
plt.ylabel("Error Value")
plt.title("Constant Acceleration Model Prediction Error (ADE & FDE)")

# Add legend with color distinction
plt.legend([
    plt.Rectangle((0, 0), 1, 1, fc="blue", alpha=0.5),
    plt.Rectangle((0, 0), 1, 1, fc="skyblue", alpha=0.5),
    plt.Rectangle((0, 0), 1, 1, fc="red", alpha=0.5),
    plt.Rectangle((0, 0), 1, 1, fc="pink", alpha=0.5)
], ["ADE Tracks", "ADE SDC", "FDE Tracks", "FDE SDC"], loc="upper left")

plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


Briefly discuss the observations from your results. 


<!-- # 车子会拐弯近停车场，但是不是所有停车场都有lane_polylines进入的，比如c496f211b25122b1就无。及时做真值也不会得到正确结果，而预测更加不会考虑到这个结果。
# 行人没有轨迹lane_polylines，所以会出现比较大的偏差，似乎没有办法进行纠正？c496f211b25122b1有corner case。 -->

In [ ]:
if __name__ == "__main__":
    cv_model = CVMScenarioVisualizer(evaluation_data_path)
    cv_results, cv_ground_truths = cv_model.process_scenarios()
    cv_evaluator = CVTrajectoryEvaluator()
    cv_evaluator.plot_evaluate_result(cv_results, cv_ground_truths, evaluation_data_path)

    ca_model = CAMScenarioVisualizer(evaluation_data_path)
    ca_results, ca_ground_truths = ca_model.process_scenarios()
    ca_evaluator = CATrajectoryEvaluator()
    ca_evaluator.plot_evaluate_result(ca_results, ca_ground_truths, evaluation_data_path)